<a href="https://colab.research.google.com/github/Pablobarchiesi94/PI_1_MLOps/blob/main/ETL/Limpieza_movies_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importo las librerias que voy a utilizar

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import ast
import json

Me conecto con Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cargo el Dataset

In [ ]:
ruta_archivo = '/content/drive/MyDrive/Dataset/movies_dataset.csv'
df= pd.read_csv(ruta_archivo)

# **Limpieza de datos**

1. Elimino las columnas que me piden en las especificaciones del proyecto

In [ ]:
columnas_descartables = ['video','imdb_id','adult','original_title','poster_path','homepage']
df = df.drop(columns=columnas_descartables)

KeyError: ignored

2. Los valores nulos de los campos revenue y budget se rellenan con ceros.

In [ ]:
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

3. Creo la columna release_year con el formato aaaa-mm-dd

In [ ]:
# Borro los valores nulos

df = df.dropna(subset=['release_date'])

In [ ]:
# Convierto a formato datatime

df['release_date'] = pd.to_datetime(df['release_date'],format='%Y-%m-%d',errors='coerce')

In [ ]:
# Compruebo si tiene valores nulos

df['release_date'].isna().sum()

0

In [ ]:
# Borro los valores nulos

df = df.dropna(subset=['release_date'])

In [ ]:
# Compruebo si hay valores nulo

df['release_date'].isna().sum()

0

In [ ]:
# Creó la columna release_year con formato datatime

df['release_year'] = df['release_date'].dt.year

4. Crear la columna return(retorno de inversion), dividiendo los campos revenue/budget, cuando no hay datos disponibles para calcular, debera tomar el valor 0.

In [ ]:
# Paso de dato type object a float

df['budget'] = pd.to_numeric(df['budget'], downcast='float', errors='coerce')

In [ ]:
# Divido los campos revenue/budget

df['return'] = df['revenue'].divide(df['budget']).fillna(0)

In [ ]:
# Cambio los valores infinito por NAN

df['return'].replace([np.inf,-np.inf], np.nan, inplace=True)

In [ ]:
# Cambio los NAN por cero

df['return'].fillna(0, inplace = True)

5. Algunos campos estan anidados deberan ser desanidados para unirlos al dataset.


In [ ]:
# Desanido listas

def fetch_name(obj):
    if isinstance(obj,str) and '{' in obj:
        s = []
        for i in ast.literal_eval(obj):
            s.append(i['name']);
        return s

In [ ]:
# Desanido diccionarios

def fetch_name1(obj):
    if isinstance(obj,str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']

In [ ]:
df['genres'] = df['genres'].apply(fetch_name)
df['belongs_to_collection'] = df['belongs_to_collection'].apply(fetch_name1)
df['production_companies'] = df['production_companies'].apply(fetch_name)
df['production_countries'] = df['production_countries'].apply(fetch_name)
df['spoken_languages'] = df['spoken_languages'].apply(fetch_name)

KeyError: ignored

In [ ]:
# Dataset con datos limpios

df.head(5)

,budget,genres,id,popularity,release_date,revenue,title,vote_average,vote_count,release_year,return,DiaSemana,Mes
0,30000000.0,None,862,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,1995,12.451801,Lunes,Octubre
1,65000000.0,None,8844,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,1995,4.043035,Viernes,Diciembre
2,0.0,None,15602,11.712900,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,1995,0.000000,Viernes,Diciembre
3,16000000.0,None,31357,3.859495,1995-12-22,81452156.0,Waiting to Exhale,6.1,34.0,1995,5.090760,Viernes,Diciembre
4,0.0,None,11862,8.387519,1995-02-10,76578911.0,Father of the Bride Part II,5.7,173.0,1995,0.000000,Viernes,Febrero


Agrego una nueva columna 'DiaSemana' que contiene el día de la semana correspondiente

In [ ]:
df['DiaSemana'] = df['release_date'].dt.day_name()

Reemplazo los nombres de los días en inglés por los correspondientes en español

In [ ]:
nombres_dias_ingles = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
nombres_dias_espanol = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
df['DiaSemana'] = df['DiaSemana'].replace(nombres_dias_ingles, nombres_dias_espanol)

Agrego una nueva columna 'Mes' que contiene el nombre del mes correspondiente en español

In [ ]:
df['Mes'] = df['release_date'].dt.month_name()

Reemplazo los nombres de los meses en inglés por los meses en español

In [ ]:
nombres_meses_ingles = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
nombres_meses_espanol = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
df['Mes'] = df['Mes'].replace(nombres_meses_ingles, nombres_meses_espanol)

In [ ]:
df.head(5)

,budget,genres,id,popularity,release_date,revenue,title,vote_average,vote_count,release_year,return,DiaSemana,Mes
0,30000000.0,None,862,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,1995,12.451801,Lunes,Octubre
1,65000000.0,None,8844,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,1995,4.043035,Viernes,Diciembre
2,0.0,None,15602,11.712900,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,1995,0.000000,Viernes,Diciembre
3,16000000.0,None,31357,3.859495,1995-12-22,81452156.0,Waiting to Exhale,6.1,34.0,1995,5.090760,Viernes,Diciembre
4,0.0,None,11862,8.387519,1995-02-10,76578911.0,Father of the Bride Part II,5.7,173.0,1995,0.000000,Viernes,Febrero


Elimino las columnas que no voy a utilizar

In [ ]:


columns_to_drop = ['belongs_to_collection', 'original_language', 'overview', 'production_companies', 'production_countries', 'runtime', 'spoken_languages', 'status', 'tagline']
df= df.drop(columns=columns_to_drop)

KeyError: ignored

Compruebo que las columnas se hayan borrado

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45376 entries, 0 to 45375
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   budget        45376 non-null  float64       
 1   genres        0 non-null      object        
 2   id            45376 non-null  int64         
 3   popularity    45376 non-null  float64       
 4   release_date  45376 non-null  datetime64[ns]
 5   revenue       45376 non-null  float64       
 6   title         45376 non-null  object        
 7   vote_average  45376 non-null  float64       
 8   vote_count    45376 non-null  float64       
 9   release_year  45376 non-null  int64         
 10  return        45376 non-null  float64       
 11  DiaSemana     45376 non-null  object        
 12  Mes           45376 non-null  object        
dtypes: datetime64[ns](1), float64(6), int64(2), object(4)
memory usage: 4.5+ MB


Guardo el Dataset limpio

In [ ]:
# Guardo

df.to_csv('df_movies_limpio.csv', index=False)

In [ ]:
# Guardo en Google Drive nuestro Dataframe limpio
df.to_csv('/content/drive/MyDrive/Dataset/movies_dataset.csv', index=False)